In [1]:
from bokeh.io import output_notebook
import pandas as pd
import numpy as np

output_notebook()

Loading BokehJS ...

In [2]:
file = "../data/facebook.csv"

df = pd.read_csv(file)

df.head()

/var/folders/yw/m_924cdx5sv965q742rtv_p40000gn/T/ipykernel_91180/3478538843.py:3: DtypeWarning: Columns (10,11,12,13,15,16,18,19,21,22,23,24,26,45,46,51,52,62,63,64,65,69,71,73,78,80,82,83,85,93,95,97,98,99,100,102,104,105,106,108,110,111,113,115,116,117,118,119,121,122,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


,Date,Post ID,Network,Post Type,Content Type,Profile,Sent by,Link,Post,Linked Content,...,Video Removed from Playlists,Annotation Impressions,Annotation Clickable Impressions,Annotation Closable Impressions,Annotation Closes,Card Impressions,Card Teaser Impressions,Card Teaser Clicks,Poll Votes,Tags
0,12/17/2022 5:08 pm,253788558082460_5441020446025886,Facebook,Post,Video,Stanbic IBTC,Damilare Oyekanmi,https://www.facebook.com/253788558082460/posts...,"We celebrated recently with Novare, one of our...",https://www.facebook.com/StanbicIBTC/videos/53...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CIB Posts
1,5/4/2019 10:01 am,253788558082460_2001824979945467,Facebook,Post,Photo,Stanbic IBTC,,https://www.facebook.com/253788558082460/posts...,N5k can get you started today. Call 01 280 126...,https://www.facebook.com/StanbicIBTC/photos/a....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6/2/2020 9:00 pm,253788558082460_2768996106561680,Facebook,Post,Photo,Stanbic IBTC,,https://www.facebook.com/StanbicIBTC/photos/a....,Still not sure whether to invest in the FGN Bo...,https://www.facebook.com/StanbicIBTC/photos/a....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5/9/2022 10:37 am,253788558082460_4808157902645480,Facebook,Post,Photo,Stanbic IBTC,Damilare Oyekanmi,https://www.facebook.com/StanbicIBTC/photos/a....,Let nothing stop you from achieving your goals...,https://www.facebook.com/StanbicIBTC/photos/a....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7/18/2021 10:00 am,253788558082460_3890983261029620,Facebook,Post,Photo,Stanbic IBTC,,https://www.facebook.com/StanbicIBTC/photos/a....,"If you were to pick one wish, what would it be...",https://www.facebook.com/StanbicIBTC/photos/a....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def clean_df(df):
    df["Time"] = pd.to_datetime(df["Date"]).dt.time
    df["Date"] = pd.to_datetime(df["Date"]).dt.date
    df = df.set_index("Date")
    for col in df.columns:
        if df[f"{col}"].value_counts().sum() == 0:
            df = df.drop(col, axis=1)

    def convert_numbers_to_float(s):
        if isinstance(s, str):
            try:
                return float(s.replace(",", ""))
            except ValueError:
                pass
        return s

    df = df.applymap(convert_numbers_to_float)

    return df

In [5]:
new_df = clean_df(df)

new_df.head()

/var/folders/yw/m_924cdx5sv965q742rtv_p40000gn/T/ipykernel_91180/2696417221.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Time"] = pd.to_datetime(df["Date"]).dt.time
/var/folders/yw/m_924cdx5sv965q742rtv_p40000gn/T/ipykernel_91180/2696417221.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"]).dt.date


,Post ID,Network,Post Type,Content Type,Profile,Sent by,Link,Post,Linked Content,Impressions,...,Paid Video View Time (Seconds),Unique Video Views,Unique Organic Video Views,Unique Paid Video Views,Unique 10-Second Video Views,Unique Full Video Views,Unique Organic 95% Video Views,Unique Paid 95% Video Views,Tags,Time
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-17,2.537886e+30,Facebook,Post,Video,Stanbic IBTC,Damilare Oyekanmi,https://www.facebook.com/253788558082460/posts...,"We celebrated recently with Novare, one of our...",https://www.facebook.com/StanbicIBTC/videos/53...,207378.0,...,0.0,975.0,975.0,0.0,263.0,62.0,19.0,0.0,CIB Posts,17:08:00
2019-05-04,2.537886e+30,Facebook,Post,Photo,Stanbic IBTC,,https://www.facebook.com/253788558082460/posts...,N5k can get you started today. Call 01 280 126...,https://www.facebook.com/StanbicIBTC/photos/a....,125784.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:01:00
2020-06-02,2.537886e+30,Facebook,Post,Photo,Stanbic IBTC,,https://www.facebook.com/StanbicIBTC/photos/a....,Still not sure whether to invest in the FGN Bo...,https://www.facebook.com/StanbicIBTC/photos/a....,89699.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21:00:00
2022-05-09,2.537886e+30,Facebook,Post,Photo,Stanbic IBTC,Damilare Oyekanmi,https://www.facebook.com/StanbicIBTC/photos/a....,Let nothing stop you from achieving your goals...,https://www.facebook.com/StanbicIBTC/photos/a....,79118.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:37:00
2021-07-18,2.537886e+30,Facebook,Post,Photo,Stanbic IBTC,,https://www.facebook.com/StanbicIBTC/photos/a....,"If you were to pick one wish, what would it be...",https://www.facebook.com/StanbicIBTC/photos/a....,76759.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:00:00
